In [1]:
import pandas as pd

In [2]:
df = {}
for k in ["train", "val", "test"]:
    df[k] = {}
    for c in ["data", "bong"]:
        df[k][c] = pd.read_excel(f"setting/{c}_v2_{k}.xlsx", index_col=0)

In [3]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

In [4]:
X_train = df["train"]["bong"]
X_val = df["val"]["bong"]
X_test = df["test"]["bong"]

X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_val = X_val.loc[:, ~X_val.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

y_train = (df["train"]["data"]["Final decision"] == "INCLUDE").apply(int)
y_val = (df["val"]["data"]["Final decision"] == "INCLUDE").apply(int)
y_test = (df["test"]["data"]["Final decision"] == "INCLUDE").apply(int)

In [5]:
import numpy as np
import random

In [6]:
train_test_sample = pd.concat([
    pd.concat([X_train, y_train], axis=1), 
    pd.concat([X_val, y_val], axis=1)
    ], ignore_index=True)
train_test_sample

,bong_1,bong_2,bong_3,bong_4,bong_5,bong_6,bong_7,bong_8,bong_9,bong_10,...,bong_992,bong_993,bong_994,bong_995,bong_996,bong_997,bong_998,bong_999,bong_1000,Final decision
0,0.0,0.0,0.076142,0.000000,0.00000,0.000000,0.000000,0.000000,0.07882,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.000000,0.047243,0.09182,0.048532,0.000000,0.041792,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.065172,0.0,0.0,0.0,1
2,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.06129,0.0,0.000000,0.000000,0.0,0.0,0.0,0
3,0.0,0.0,0.000000,0.000000,0.00000,0.055916,0.057194,0.000000,0.00000,0.0,...,0.045521,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.052758,0.000000,0.0,0.0,0.0,0
1596,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.039327,0.0,0.0,0.0,1
1597,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.076594,0.0,0.0,0.0,0
1598,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0


In [7]:
F_features = np.load("output/bong_features.npy")

In [8]:
features, counts = np.unique(F_features, return_counts=True)
k = 5
new_best_features = features[counts>=k]

X_train_val = train_test_sample.drop(columns="Final decision")
y_train_val = train_test_sample["Final decision"]

clf = DecisionTreeClassifier(random_state=42, max_depth=5)
clf.fit(X_train_val[new_best_features], y_train_val)
print(clf.score(X_train_val[new_best_features], y_train_val))
print(classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4))
o = classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4, output_dict=True)
o

0.7575
              precision    recall  f1-score   support

           0     0.6453    0.5904    0.6167       188
           1     0.6623    0.7123    0.6864       212

    accuracy                         0.6550       400
   macro avg     0.6538    0.6513    0.6515       400
weighted avg     0.6543    0.6550    0.6536       400



{'0': {'precision': 0.6453488372093024,
  'recall': 0.5904255319148937,
  'f1-score': 0.6166666666666667,
  'support': 188},
 '1': {'precision': 0.6622807017543859,
  'recall': 0.7122641509433962,
  'f1-score': 0.6863636363636364,
  'support': 212},
 'accuracy': 0.655,
 'macro avg': {'precision': 0.6538147694818441,
  'recall': 0.651344841429145,
  'f1-score': 0.6515151515151516,
  'support': 400},
 'weighted avg': {'precision': 0.6543227254181966,
  'recall': 0.655,
  'f1-score': 0.6536060606060607,
  'support': 400}}